In [37]:
import random
import csv
import time, array

import pandas as pd
from math import factorial

from deap import base, creator, tools, algorithms, benchmarks 

from scipy.spatial.distance import cdist





import random
from math import sin, cos, pi, exp, e, sqrt
from operator import mul
from functools import reduce

from deap import base, creator, tools, algorithms, benchmarks 

import numpy as np
import autograd.numpy as anp
import pandas as pd

import pymop
from deap.benchmarks.tools import diversity, convergence, hypervolume

import csv
import time, array

import matplotlib.pyplot as plt
import seaborn as sns

from mpl_toolkits.mplot3d import Axes3D

from math import factorial

In [38]:
def mutInversion(individual):
    size = len(individual)
    if size == 0:
        return individual,

    index_one = random.randrange(size)
    index_two = random.randrange(size)
    start_index = min(index_one, index_two)
    end_index = max(index_one, index_two)

    # Reverse the contents of the individual between the indices
    individual[start_index:end_index] = individual[start_index:end_index][::-1]

    return individual,

In [39]:
def measure_time(x):
    execution_time = time.time() - start_time 
#     print(execution_time)
    return execution_time

In [40]:
def nsga_ii(toolbox, seed, stats=None, verbose=False):
    pop = toolbox.population(n=toolbox.pop_size)
    pop = toolbox.select(pop, len(pop))
    random.seed(seed)
    return algorithms.eaMuPlusLambda(pop, 
                                     toolbox, 
                                     mu=toolbox.pop_size, 
                                     lambda_=toolbox.pop_size, 
                                     cxpb=toolbox.cross_prob,
                                     mutpb=toolbox.mut_prob,
                                     ngen=toolbox.max_gen,
                                     stats=stats, 
                                     verbose=verbose)

In [77]:
def nsga_iii(toolbox,stats,logbook,NGEN,MU,CXPB,MUTPB,verbose=False):
    pop = toolbox.population(n=MU)
    invalid_ind = [ind for ind in pop if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
#   fitnesses = toolbox.map(toolbox.evaluate, pop)        
#     for ind, fit in zip(pop, fitnesses):
#         ind.fitness.values = fit
    
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    pop = toolbox.select(pop, len(pop))
    
    # Begin the generational process
    for gen in range(1, NGEN):
        offspring = algorithms.varAnd(pop, toolbox, CXPB, MUTPB)

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        record = stats.compile(pop) if stats is not None else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        
        if verbose:
            print(logbook.stream)
        
        # Select the next generation population
        pop = toolbox.select(pop + offspring, MU)
    
    return pop, logbook

In [69]:
kroA100=pd.read_csv('C:/Users/Maria/Documents/Python Scripts/kroA100.csv', header=0, encoding='utf-8', skip_blank_lines=False, delimiter=r",")
kroB100=pd.read_csv('C:/Users/Maria/Documents/Python Scripts/kroB100.csv', header=0, encoding='utf-8', skip_blank_lines=False, delimiter=r",")
kroC100=pd.read_csv('C:/Users/Maria/Documents/Python Scripts/kroC100.csv', header=0, encoding='utf-8', skip_blank_lines=False, delimiter=r",")

In [70]:
one = cdist(kroA100.values,kroA100.values)
two = cdist(kroB100.values,kroB100.values)
three = cdist(kroC100.values,kroC100.values)

In [71]:
def motsp(x, one=one, two=two, three=three):
        n_cities = len(x)
        dist_one = 0
        dist_two = 0
        dist_three = 0
        
        for k in range(n_cities - 1):
            i, j = x[k], x[k + 1]
            dist_one += one[i, j]
            dist_two += two[i, j]
            dist_three += three[i, j]

        last, first = x[-1], x[0]
        
        dist_one += one[last, first]
        dist_one = 100000/dist_one
        dist_two += two[last, first]
        dist_two = 100000/dist_two
        dist_three += three[last, first]
        dist_three = 100000/dist_three
        
        return dist_one, dist_two, dist_three

In [72]:
NOBJ = 3
NGEN = 5000
CXPB = 0.8
MUTPB = 0.2

P = 12
H = factorial(NOBJ + P - 1) / (factorial(P) * factorial(NOBJ - 1))
ref_points = tools.uniform_reference_points(NOBJ, P)

MU = 92

creator.create("FitnessMin", base.Fitness, weights=(-1.0,) * NOBJ) 
creator.create("Individual", list, fitness=creator.FitnessMin)

IND_SIZE=100

toolbox = base.Toolbox()
toolbox.register("indices", random.sample, range(100), IND_SIZE)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", motsp) #Fitness Function

toolbox.register("mate", tools.cxOrdered)
toolbox.register("mutate", mutInversion)
toolbox.register("select", tools.selNSGA2)

toolbox.pop_size = MU
toolbox.max_gen = NGEN
toolbox.mut_prob = MUTPB
toolbox.cross_prob = CXPB

# Initialize statistics object
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)
stats.register("time", measure_time)

logbook = tools.Logbook()
logbook.header = "gen", "evals", "std", "min", "avg", "max", "time"

C:\Users\Maria\miniconda3\envs\fs\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\Maria\miniconda3\envs\fs\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [47]:
start_time = time.time()
res, logbook = nsga_ii(toolbox=toolbox, seed=1, stats=stats, verbose=True)

fronts = tools.emo.sortLogNondominated(res, len(res))

for i, inds in enumerate(fronts):
    par = [toolbox.evaluate(ind) for ind in inds]
    df = pd.DataFrame(par)

df.to_csv('deap_fitness_nsga2_motsp_92_500_3_100.csv', index=False)
df = pd.DataFrame(logbook)
df.to_csv('deap_time_nsga2_motsp_92_500_3_100.csv', index=False)

gen	nevals	avg     	std      	min     	max     	time     
0  	92    	0.587443	0.0272071	0.522833	0.681525	0.0358565
1  	92    	0.576058	0.0268398	0.517879	0.681525	0.200765 
2  	92    	0.570097	0.0269529	0.515754	0.677256	0.35188  
3  	92    	0.564967	0.0265004	0.51561 	0.677256	0.495788 
4  	92    	0.561294	0.0244776	0.507412	0.648719	0.637785 
5  	92    	0.559016	0.0264221	0.504475	0.659351	0.778827 
6  	92    	0.556968	0.0296091	0.503808	0.659351	0.882737 
7  	92    	0.554348	0.0282413	0.503363	0.657957	0.977048 
8  	92    	0.551459	0.0277132	0.501361	0.638532	1.11468  
9  	92    	0.547488	0.0268145	0.490653	0.62214 	1.22671  
10 	92    	0.54512 	0.0275146	0.490653	0.62214 	1.33436  
11 	92    	0.544598	0.0285525	0.490653	0.62214 	1.44385  
12 	92    	0.543793	0.0297369	0.490653	0.62214 	1.59911  
13 	92    	0.543265	0.0307432	0.490653	0.634273	1.79792  
14 	92    	0.541908	0.0322471	0.490653	0.627229	1.96231  
15 	92    	0.541   	0.0325126	0.490653	0.627382	2.13503  
16 	92    	0.5

142	92    	0.494461	0.0342675	0.436366	0.582206	19.1707  
143	92    	0.493848	0.0332928	0.436366	0.582206	19.2549  
144	92    	0.493827	0.0337629	0.436366	0.582206	19.3607  
145	92    	0.493835	0.0340108	0.436366	0.582206	19.4406  
146	92    	0.493389	0.0331391	0.435412	0.582206	19.5457  
147	92    	0.493563	0.0335451	0.435412	0.582206	19.68    
148	92    	0.493894	0.0343581	0.435412	0.582206	19.8059  
149	92    	0.493844	0.0339878	0.435412	0.582206	19.9496  
150	92    	0.493891	0.0343263	0.435412	0.582206	20.0981  
151	92    	0.494215	0.0355589	0.435412	0.588973	20.2173  
152	92    	0.494003	0.0354081	0.435412	0.588973	20.3548  
153	92    	0.49368 	0.0355523	0.435412	0.588973	20.4957  
154	92    	0.493435	0.0354897	0.435412	0.588973	20.5958  
155	92    	0.493588	0.0353677	0.435412	0.588973	20.6969  
156	92    	0.493593	0.03558  	0.435412	0.588973	20.7846  
157	92    	0.493646	0.0356453	0.435412	0.588973	20.8538  
158	92    	0.493698	0.0353602	0.435412	0.588973	20.9536  
159	92    	0.4

286	92    	0.489606	0.0413605	0.425921	0.607037	37.5456  
287	92    	0.489507	0.0411366	0.425921	0.607037	37.6397  
288	92    	0.489093	0.0417325	0.425795	0.607037	37.7391  
289	92    	0.488782	0.0413112	0.425795	0.607037	37.8627  
290	92    	0.488752	0.0408335	0.425788	0.607037	38.0356  
291	92    	0.489331	0.0414399	0.425788	0.607037	38.2247  
292	92    	0.48923 	0.0411623	0.42549 	0.607037	38.3673  
293	92    	0.489725	0.0415561	0.42549 	0.607037	38.501   
294	92    	0.489255	0.0416387	0.42549 	0.607037	38.6738  
295	92    	0.488784	0.0410885	0.425463	0.607037	38.8479  
296	92    	0.489182	0.0410082	0.425463	0.607037	38.9909  
297	92    	0.488656	0.0404007	0.425463	0.607037	39.1652  
298	92    	0.488247	0.0396138	0.425463	0.594612	39.286   
299	92    	0.488214	0.0398314	0.425463	0.594612	39.4118  
300	92    	0.488973	0.0413037	0.423302	0.594612	39.5302  
301	92    	0.48892 	0.0414323	0.423302	0.594612	39.6337  
302	92    	0.48912 	0.0415484	0.423302	0.591647	39.7209  
303	92    	0.4

429	92    	0.49015 	0.0469538	0.421904	0.602267	57.022   
430	92    	0.49014 	0.0470659	0.421904	0.602267	57.1366  
431	92    	0.490237	0.0458145	0.421904	0.602267	57.2428  
432	92    	0.490385	0.0450725	0.421904	0.602267	57.4368  
433	92    	0.490289	0.0455025	0.421904	0.603347	57.6247  
434	92    	0.490013	0.0453667	0.421904	0.603347	57.7698  
435	92    	0.490421	0.0449394	0.421904	0.603347	57.9835  
436	92    	0.490707	0.04618  	0.421904	0.603347	58.173   
437	92    	0.490217	0.0459054	0.421904	0.603347	58.3437  
438	92    	0.490317	0.0460058	0.421904	0.603347	58.5346  
439	92    	0.490698	0.0471461	0.421904	0.603347	58.709   
440	92    	0.490893	0.0475365	0.421904	0.603347	58.8828  
441	92    	0.490625	0.0468992	0.421904	0.603347	59.1136  
442	92    	0.490584	0.0462533	0.421904	0.603347	59.2471  
443	92    	0.490843	0.0459255	0.421904	0.603347	59.3651  
444	92    	0.490161	0.0456297	0.421904	0.603347	59.5676  
445	92    	0.489727	0.0452496	0.421904	0.603347	59.6937  
446	92    	0.4

573	92    	0.496972	0.057389 	0.416483	0.658444	76.9619  
574	92    	0.497362	0.0580189	0.416483	0.658444	77.1371  
575	92    	0.497338	0.0579701	0.416483	0.658444	77.2584  
576	92    	0.496852	0.0584185	0.416483	0.658444	77.3818  
577	92    	0.496006	0.0569384	0.416483	0.658444	77.5205  
578	92    	0.49652 	0.0570796	0.416483	0.658444	77.6976  
579	92    	0.496102	0.0557881	0.416483	0.658444	77.8404  
580	92    	0.496006	0.0557995	0.416483	0.658444	77.9461  
581	92    	0.496039	0.0560292	0.416399	0.658444	78.0955  
582	92    	0.495752	0.0559991	0.416399	0.658444	78.2616  
583	92    	0.49555 	0.055887 	0.416399	0.658444	78.4154  
584	92    	0.496144	0.0557522	0.416399	0.658444	78.524   
585	92    	0.495693	0.0560235	0.416399	0.658444	78.6676  
586	92    	0.49601 	0.0572188	0.416399	0.658444	78.8359  
587	92    	0.496113	0.05678  	0.416399	0.658444	79.0131  
588	92    	0.496084	0.0571406	0.416399	0.658444	79.1411  
589	92    	0.496543	0.0571249	0.416399	0.658444	79.2451  
590	92    	0.4

...

4355	92    	0.515893	0.0770008	0.39866 	0.715689	534.989  
4356	92    	0.515407	0.0771898	0.39866 	0.715689	535.104  
4357	92    	0.514702	0.0758322	0.39866 	0.715689	535.208  
4358	92    	0.514008	0.0757529	0.39866 	0.703777	535.326  
4359	92    	0.514572	0.0759565	0.39866 	0.703777	535.432  
4360	92    	0.514858	0.0759295	0.39866 	0.703777	535.572  
4361	92    	0.515213	0.0768364	0.39866 	0.703777	535.652  
4362	92    	0.514509	0.0757757	0.39866 	0.703777	535.717  
4363	92    	0.514079	0.0754938	0.39866 	0.703777	535.82   
4364	92    	0.514358	0.0770563	0.39866 	0.703777	535.968  
4365	92    	0.513834	0.0748353	0.39866 	0.703777	536.081  
4366	92    	0.513968	0.0752559	0.39866 	0.703777	536.2    
4367	92    	0.51396 	0.0757945	0.39866 	0.703777	536.302  
4368	92    	0.514469	0.0758993	0.39866 	0.703777	536.401  
4369	92    	0.514217	0.0755035	0.39866 	0.703777	536.501  
4370	92    	0.514777	0.0753323	0.39866 	0.703777	536.601  
4371	92    	0.51445 	0.0746964	0.39866 	0.703893	536.716

4495	92    	0.515129	0.0744649	0.398658	0.699357	550.229  
4496	92    	0.515401	0.0739894	0.398658	0.699357	550.342  
4497	92    	0.515699	0.0735062	0.398658	0.699357	550.435  
4498	92    	0.515888	0.0748001	0.398658	0.699357	550.536  
4499	92    	0.516572	0.0742381	0.398658	0.699357	550.653  
4500	92    	0.517064	0.0750208	0.398639	0.699357	550.799  
4501	92    	0.517146	0.0754448	0.398639	0.699357	550.901  
4502	92    	0.5166  	0.0761012	0.398639	0.699357	551.073  
4503	92    	0.516534	0.0758513	0.398639	0.699357	551.175  
4504	92    	0.517053	0.0769486	0.398639	0.699357	551.277  
4505	92    	0.517118	0.0765714	0.398639	0.699357	551.381  
4506	92    	0.517588	0.0760974	0.398639	0.699357	551.48   
4507	92    	0.517177	0.0756339	0.398639	0.699357	551.601  
4508	92    	0.517664	0.0764574	0.398639	0.699357	551.723  
4509	92    	0.517794	0.0756393	0.398639	0.699357	551.826  
4510	92    	0.517559	0.0758679	0.398639	0.699357	551.931  
4511	92    	0.517589	0.0756576	0.398639	0.699357	552.091

4635	92    	0.517537	0.0768286	0.398581	0.692995	567.359  
4636	92    	0.518032	0.0758398	0.398581	0.692995	567.569  
4637	92    	0.518571	0.0767567	0.398581	0.692995	567.778  
4638	92    	0.519026	0.0771622	0.398581	0.692995	567.965  
4639	92    	0.519863	0.0784063	0.398581	0.692995	568.093  
4640	92    	0.519496	0.0787065	0.398581	0.692995	568.207  
4641	92    	0.518744	0.0773969	0.398581	0.692995	568.324  
4642	92    	0.518745	0.0784766	0.398581	0.692995	568.423  
4643	92    	0.51852 	0.0772465	0.398581	0.692995	568.537  
4644	92    	0.518846	0.0776499	0.398581	0.692995	568.643  
4645	92    	0.519057	0.077487 	0.398581	0.692995	568.744  
4646	92    	0.519217	0.0775557	0.398581	0.692995	568.851  
4647	92    	0.519604	0.0775516	0.398581	0.692995	568.97   
4648	92    	0.519864	0.0787276	0.398581	0.694719	569.085  
4649	92    	0.52028 	0.078473 	0.398581	0.694719	569.187  
4650	92    	0.519993	0.0796005	0.398581	0.694719	569.288  
4651	92    	0.518978	0.0781166	0.398581	0.69128 	569.388

4776	92    	0.518481	0.0725324	0.398426	0.703482	587.62   
4777	92    	0.519009	0.074023 	0.398426	0.717035	587.74   
4778	92    	0.518669	0.0733461	0.398426	0.717035	587.85   
4779	92    	0.518199	0.0736467	0.398426	0.717035	587.973  
4780	92    	0.518332	0.0733314	0.398426	0.717035	588.093  
4781	92    	0.518107	0.0731011	0.398426	0.717035	588.202  
4782	92    	0.517899	0.0743508	0.398426	0.717035	588.321  
4783	92    	0.517378	0.0747381	0.398426	0.717035	588.443  
4784	92    	0.516039	0.0730191	0.398426	0.717035	588.553  
4785	92    	0.515688	0.0716956	0.398426	0.717035	588.686  
4786	92    	0.51524 	0.0719845	0.398349	0.717035	588.817  
4787	92    	0.514984	0.0707341	0.398349	0.717035	589      
4788	92    	0.515118	0.071747 	0.398349	0.717035	589.126  
4789	92    	0.515911	0.0725928	0.398328	0.717035	589.265  
4790	92    	0.516133	0.0726285	0.398328	0.717035	589.385  
4791	92    	0.516263	0.0730675	0.398328	0.717035	589.513  
4792	92    	0.516404	0.0725989	0.398328	0.717035	589.608

4915	92    	0.515162	0.0766262	0.398302	0.715004	607.052  
4916	92    	0.516552	0.0764496	0.398302	0.715004	607.164  
4917	92    	0.516212	0.0757052	0.398302	0.715004	607.291  
4918	92    	0.516329	0.077463 	0.398302	0.715004	607.458  
4919	92    	0.515684	0.0763466	0.398302	0.715004	607.567  
4920	92    	0.515606	0.0769522	0.398302	0.715004	607.672  
4921	92    	0.515228	0.0766559	0.398302	0.715004	607.786  
4922	92    	0.51585 	0.0764711	0.398302	0.715004	607.892  
4923	92    	0.514028	0.0746742	0.398302	0.715004	608      
4924	92    	0.512851	0.0728736	0.398302	0.715004	608.088  
4925	92    	0.512983	0.0745396	0.398302	0.715004	608.18   
4926	92    	0.512793	0.073629 	0.398302	0.715004	608.273  
4927	92    	0.513307	0.0742136	0.398263	0.715004	608.392  
4928	92    	0.513128	0.0749297	0.398263	0.715004	608.497  
4929	92    	0.513143	0.0745528	0.398263	0.715004	608.603  
4930	92    	0.513128	0.0743771	0.398263	0.715004	608.713  
4931	92    	0.513786	0.0745812	0.398263	0.715004	608.834

In [81]:
NOBJ = 3
NGEN = 5000
CXPB = 0.8
MUTPB = 0.2

P = 12
H = factorial(NOBJ + P - 1) / (factorial(P) * factorial(NOBJ - 1))

MU = int(H + (4 - H % 4))

ref_points = tools.uniform_reference_points(NOBJ, P)

creator.create("FitnessMin", base.Fitness, weights=(-1.0,) * NOBJ) 
creator.create("Individual", list, fitness=creator.FitnessMin)

IND_SIZE=100

toolbox = base.Toolbox()
toolbox.register("indices", random.sample, range(100), IND_SIZE)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", motsp) #Fitness Function

toolbox.register("mate", tools.cxOrdered)
toolbox.register("mutate", mutInversion)
toolbox.register("select", tools.selNSGA3, ref_points=ref_points)

# Initialize statistics object
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)
stats.register("time", measure_time)

logbook = tools.Logbook()
logbook.header = "gen", "evals", "std", "min", "avg", "max", "time"

C:\Users\Maria\miniconda3\envs\fs\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\Maria\miniconda3\envs\fs\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [82]:
start_time = time.time()
res, logbook = nsga_iii(toolbox=toolbox,stats=stats,logbook=logbook,NGEN=NGEN,MU=MU,CXPB=CXPB,MUTPB=MUTPB,verbose=True)

fronts = tools.emo.sortLogNondominated(res, len(res))

for i, inds in enumerate(fronts):
    par = [toolbox.evaluate(ind) for ind in inds]
    df = pd.DataFrame(par)

df.to_csv('deap_fitness_nsga3_motsp_92_500_3_100.csv', index=False)

df = pd.DataFrame(logbook)
df.to_csv('deap_time_nsga3_motsp_92_500_3_100.csv', index=False)

gen	evals	std      	min     	avg     	max     	time     
1  	     	0.0290811	0.518277	0.587906	0.687894	0.0733404
2  	     	0.0283726	0.518277	0.577011	0.667322	0.116098 
3  	     	0.0299631	0.507816	0.57044 	0.667322	0.159028 
4  	     	0.030812 	0.507816	0.565305	0.667013	0.194286 
5  	     	0.0312326	0.501987	0.56106 	0.667013	0.241755 
6  	     	0.0344649	0.49945 	0.558614	0.667013	0.290766 
7  	     	0.0368608	0.490534	0.556017	0.68542 	0.334198 
8  	     	0.036935 	0.490534	0.553295	0.68542 	0.378196 
9  	     	0.0365473	0.490534	0.548948	0.68542 	0.420213 
10 	     	0.0360315	0.488537	0.544865	0.68542 	0.461034 
11 	     	0.0395681	0.488064	0.543992	0.68542 	0.505187 
12 	     	0.036006 	0.482184	0.540228	0.674379	0.548358 
13 	     	0.0358468	0.482184	0.538452	0.674379	0.591931 
14 	     	0.0334032	0.482184	0.53603 	0.674379	0.645666 
15 	     	0.03341  	0.482184	0.534156	0.674379	0.719297 
16 	     	0.0373792	0.478681	0.534656	0.668273	0.788374 
17 	     	0.0303904	0.478681	0.

148	     	0.0203418	0.433533	0.466736	0.534985	7.80358  
149	     	0.0189472	0.433533	0.465522	0.528671	7.85745  
150	     	0.0192809	0.433533	0.465742	0.528671	7.89732  
151	     	0.0188826	0.433182	0.46538 	0.528671	7.93684  
152	     	0.0197505	0.433182	0.466047	0.528671	7.98597  
153	     	0.0191771	0.433182	0.464509	0.528671	8.03186  
154	     	0.0188853	0.433182	0.464073	0.528671	8.07201  
155	     	0.0185374	0.433182	0.46396 	0.528671	8.11503  
156	     	0.0180991	0.433182	0.463582	0.528671	8.17545  
157	     	0.017873 	0.433182	0.463466	0.528671	8.23092  
158	     	0.0175418	0.433182	0.463136	0.52029 	8.27792  
159	     	0.017216 	0.433182	0.463022	0.52029 	8.32686  
160	     	0.0166043	0.433182	0.46262 	0.52029 	8.38447  
161	     	0.0174974	0.433182	0.462887	0.52029 	8.43619  
162	     	0.0176908	0.433182	0.462862	0.52029 	8.47833  
163	     	0.0175795	0.433182	0.462312	0.52029 	8.5191   
164	     	0.0170872	0.433182	0.462017	0.52029 	8.56     
165	     	0.0167484	0.433182	0.

296	     	0.0163858	0.425568	0.453001	0.518261	14.7184  
297	     	0.0164065	0.425568	0.452797	0.518261	14.7644  
298	     	0.0162797	0.425568	0.452603	0.518261	14.8069  
299	     	0.0166719	0.425568	0.453051	0.518261	14.8641  
300	     	0.0160576	0.425568	0.452545	0.499621	14.9079  
301	     	0.0158262	0.425568	0.452355	0.499621	14.9519  
302	     	0.0161727	0.425568	0.452803	0.499621	15.0019  
303	     	0.0160855	0.425568	0.452672	0.499621	15.0509  
304	     	0.0159926	0.425568	0.452492	0.499621	15.1003  
305	     	0.0159492	0.425568	0.452475	0.499988	15.1503  
306	     	0.0159696	0.425568	0.452436	0.499988	15.2023  
307	     	0.0157163	0.425568	0.452053	0.499988	15.2468  
308	     	0.0156546	0.425568	0.452169	0.499988	15.2881  
309	     	0.0164108	0.425107	0.452717	0.499988	15.3339  
310	     	0.016099 	0.425107	0.452546	0.499988	15.377   
311	     	0.0163711	0.425107	0.452454	0.499988	15.4285  
312	     	0.015977 	0.425107	0.452171	0.499988	15.4782  
313	     	0.0162425	0.425107	0.

440	     	0.0167269	0.419516	0.447862	0.508525	21.723   
441	     	0.0170164	0.419516	0.448093	0.508525	21.78    
442	     	0.0169641	0.419516	0.448167	0.508525	21.8315  
443	     	0.0167814	0.419516	0.44794 	0.508525	21.8763  
444	     	0.015973 	0.419516	0.447478	0.508525	21.9222  
445	     	0.0165729	0.419516	0.447635	0.508525	21.9679  
446	     	0.016522 	0.419516	0.44764 	0.508525	22.0158  
447	     	0.0166499	0.419516	0.447462	0.508525	22.056   
448	     	0.0161798	0.419516	0.447105	0.508525	22.1123  
449	     	0.0162176	0.419516	0.446871	0.508525	22.1585  
450	     	0.0157197	0.419516	0.446452	0.508525	22.2044  
451	     	0.0147964	0.419516	0.445807	0.508525	22.2495  
452	     	0.0148579	0.419516	0.445864	0.508525	22.2991  
453	     	0.0141729	0.421146	0.445558	0.508061	22.3444  
454	     	0.0143056	0.421146	0.445696	0.508061	22.3897  
455	     	0.0142069	0.421146	0.445683	0.508061	22.4391  
456	     	0.0141553	0.421146	0.445729	0.508061	22.4834  
457	     	0.0151508	0.420283	0.

588	     	0.0161164	0.419584	0.444202	0.511816	29.0229  
589	     	0.0167006	0.419584	0.444452	0.511816	29.0698  
590	     	0.0170796	0.419584	0.444817	0.511816	29.1129  
591	     	0.0172617	0.419584	0.444833	0.511816	29.1702  
592	     	0.0173352	0.419584	0.44489 	0.511816	29.2081  
593	     	0.0173319	0.419584	0.4449  	0.511816	29.2557  
594	     	0.0168802	0.419584	0.444597	0.511816	29.3018  
595	     	0.0167483	0.419584	0.444552	0.511816	29.3459  
596	     	0.0166767	0.419584	0.44448 	0.511816	29.3962  
597	     	0.016668 	0.419367	0.444411	0.511816	29.4446  
598	     	0.0173079	0.419367	0.444647	0.511816	29.489   
599	     	0.0170256	0.419367	0.444455	0.511816	29.5401  
600	     	0.0163806	0.419367	0.443912	0.509817	29.5872  
601	     	0.0164909	0.419367	0.443883	0.509817	29.6464  
602	     	0.0161551	0.419367	0.443743	0.509817	29.6904  
603	     	0.0160621	0.419367	0.443628	0.509817	29.7349  
604	     	0.0158964	0.419367	0.443313	0.509817	29.7869  
605	     	0.0159697	0.419367	0.

...

4424	     	0.011291  	0.408162	0.426763	0.482209	218.93   
4425	     	0.011663  	0.408162	0.426936	0.482209	218.98   
4426	     	0.0115783 	0.408162	0.426887	0.482209	219.02   
4427	     	0.0116452 	0.408162	0.42689 	0.482209	219.068  
4428	     	0.0120906 	0.408162	0.42713 	0.482209	219.133  
4429	     	0.0122489 	0.408162	0.427252	0.482209	219.182  
4430	     	0.0122972 	0.408162	0.427216	0.482209	219.228  
4431	     	0.0122684 	0.408162	0.427201	0.482209	219.293  
4432	     	0.0125033 	0.408162	0.427341	0.482209	219.33   
4433	     	0.0125114 	0.408162	0.427367	0.482209	219.386  
4434	     	0.0125033 	0.408162	0.427346	0.482209	219.435  
4435	     	0.0122616 	0.408162	0.427264	0.482209	219.487  
4436	     	0.0122221 	0.408162	0.427175	0.482209	219.529  
4437	     	0.0117434 	0.408162	0.426995	0.482209	219.57   
4438	     	0.0121067 	0.408162	0.427258	0.482209	219.61   
4439	     	0.0122562 	0.408162	0.427352	0.482209	219.669  
4440	     	0.0124417 	0.407885	0.427486	0.482209	219.715

4563	     	0.0123158 	0.408147	0.427575	0.491151	225.638  
4564	     	0.0121546 	0.408147	0.427537	0.491151	225.677  
4565	     	0.0120527 	0.408147	0.427474	0.491151	225.722  
4566	     	0.0119283 	0.408147	0.427562	0.491151	225.767  
4567	     	0.0105623 	0.408147	0.427082	0.462527	225.812  
4568	     	0.0104325 	0.408147	0.426968	0.462527	225.856  
4569	     	0.0107832 	0.408147	0.427034	0.462527	225.902  
4570	     	0.0105046 	0.408147	0.426875	0.462527	225.945  
4571	     	0.0104447 	0.408147	0.426853	0.462527	225.983  
4572	     	0.0103677 	0.408147	0.426802	0.462527	226.028  
4573	     	0.0104303 	0.408147	0.426786	0.462527	226.075  
4574	     	0.0106129 	0.408147	0.426873	0.462527	226.114  
4575	     	0.010633  	0.408147	0.426916	0.462527	226.17   
4576	     	0.010589  	0.408147	0.426813	0.462527	226.213  
4577	     	0.0104071 	0.408147	0.426686	0.462527	226.258  
4578	     	0.0104231 	0.408147	0.426614	0.462527	226.301  
4579	     	0.0103956 	0.408147	0.426583	0.462527	226.345

4706	     	0.0109149 	0.408121	0.426137	0.466969	232.411  
4707	     	0.0109591 	0.408121	0.426164	0.466969	232.457  
4708	     	0.0109185 	0.408121	0.426083	0.466969	232.498  
4709	     	0.0108441 	0.408427	0.42606 	0.466969	232.542  
4710	     	0.0108182 	0.408427	0.425998	0.466969	232.58   
4711	     	0.0110464 	0.408427	0.426076	0.466969	232.615  
4712	     	0.0107685 	0.408427	0.42596 	0.466969	232.662  
4713	     	0.0108537 	0.408427	0.425904	0.466969	232.721  
4714	     	0.0107519 	0.408427	0.425868	0.466969	232.761  
4715	     	0.010502  	0.408427	0.425597	0.466969	232.796  
4716	     	0.0106361 	0.408427	0.425725	0.466969	232.828  
4717	     	0.0105644 	0.408427	0.425675	0.466969	232.866  
4718	     	0.0105231 	0.408427	0.425644	0.466969	232.925  
4719	     	0.0106168 	0.408427	0.425643	0.466969	233.042  
4720	     	0.010624  	0.408427	0.425562	0.466969	233.114  
4721	     	0.0107367 	0.408427	0.425681	0.466969	233.2    
4722	     	0.0106761 	0.408427	0.425654	0.466969	233.276

4849	     	0.0102146 	0.409164	0.424755	0.469059	239.208  
4850	     	0.0102991 	0.409164	0.424801	0.469059	239.249  
4851	     	0.0103078 	0.40928 	0.424849	0.467858	239.295  
4852	     	0.0102746 	0.40928 	0.424845	0.467858	239.341  
4853	     	0.0096198 	0.40928 	0.424571	0.466969	239.403  
4854	     	0.0097263 	0.40928 	0.424856	0.466969	239.446  
4855	     	0.00964829	0.40928 	0.424867	0.466969	239.493  
4856	     	0.00955143	0.40928 	0.424641	0.466969	239.537  
4857	     	0.00953777	0.40928 	0.424556	0.466969	239.581  
4858	     	0.00938412	0.40928 	0.424587	0.466969	239.626  
4859	     	0.00968105	0.40928 	0.424682	0.466969	239.675  
4860	     	0.00978057	0.40928 	0.424728	0.466969	239.718  
4861	     	0.00959501	0.40928 	0.424457	0.466969	239.761  
4862	     	0.00961392	0.408353	0.424517	0.466969	239.804  
4863	     	0.0093261 	0.408353	0.424396	0.466969	239.853  
4864	     	0.0095158 	0.408353	0.424481	0.466969	239.892  
4865	     	0.00948961	0.408353	0.424472	0.466969	239.95 

4988	     	0.0105471 	0.408669	0.425275	0.467102	246.008  
4989	     	0.0105724 	0.408669	0.424999	0.467102	246.055  
4990	     	0.0106227 	0.408669	0.425163	0.467102	246.114  
4991	     	0.01053   	0.408669	0.425132	0.467102	246.158  
4992	     	0.010637  	0.408496	0.425399	0.467102	246.199  
4993	     	0.0108542 	0.408496	0.425443	0.467102	246.245  
4994	     	0.0107104 	0.408496	0.42541 	0.467102	246.288  
4995	     	0.0107401 	0.408496	0.425524	0.467102	246.333  
4996	     	0.0107546 	0.408496	0.425297	0.467102	246.377  
4997	     	0.0107727 	0.407688	0.425197	0.467102	246.412  
4998	     	0.0109731 	0.407688	0.425539	0.467102	246.469  
4999	     	0.0106061 	0.407688	0.425186	0.467102	246.519  
